An error occurred: Parameter 't' must be defined before 'st'.

Example with more parameters:
An error occurred: Parameter 'd' must be defined before 'e'.

Example with circular dependency:
Circular dependency detected: Error in topological sorting: Circular dependency detected involving 'a'.


In [5]:
my_graph = build_dependency_graph(my_parameter_space_dict)

In [6]:
topological_sort(my_graph)

['st', 't', 'sv']

In [41]:
import numpy as np
from typing import Any, Dict, List, Set, Tuple
from collections import defaultdict


def parse_bounds(bounds: Tuple[Any, Any]) -> Set[str]:
    """
    Parse the bounds of a parameter and extract any dependencies.

    Parameters:
        bounds (Tuple[Any, Any]): A tuple containing the lower and upper bounds,
                                  which can be numeric or strings indicating dependencies.

    Returns:
        Set[str]: A set of parameter names that the bounds depend on.
    """
    dependencies = set()
    for value in bounds:
        if isinstance(value, str):
            dependencies.add(value)
    return dependencies


def build_dependency_graph(
    param_dict: Dict[str, Tuple[Any, Any]]
) -> Dict[str, Set[str]]:
    """
    Build a dependency graph based on parameter bounds.

    Parameters:
        param_dict (Dict[str, Tuple[Any, Any]]): A dictionary mapping parameter names to their bounds.

    Returns:
        Dict[str, Set[str]]: A dictionary representing the dependency graph where keys are parameter names,
                             and values are sets of parameter names they depend on.
    """

    # Note: For the topological sort to work properly
    # we need to construct this graph so that
    # keys represent 'parents' and values represent sets
    # of 'children'!

    # e.g.
    # param_dict = {'a': (0, 5), 'b': (0, 'a'), 'c': ('b', 'a')}
    # resulting graph = {'a': {'b', 'c'}, 'b': {'c'}, 'c': set()}
    graph: Dict[str, Set[str]] = defaultdict(set)
    all_params = set(param_dict.keys())
    for param, bounds in param_dict.items():
        dependencies = parse_bounds(bounds)
        for dependency in dependencies:
            if dependency not in all_params:
                raise ValueError(f"Parameter '{dependency}' is not defined.")
            else:
                graph[dependency].add(param)
        all_params.update(dependencies)
    # Ensure all parameters are in the graph
    for param in all_params:
        if param not in graph:
            graph[param] = set()
    return graph


def topological_sort_util(
    node: str,
    visited: Set[str],
    stack: List[str],
    graph: Dict[str, Set[str]],
    temp_marks: Set[str],
) -> None:
    """
    Helper function for performing a depth-first search in the topological sort.

    Parameters:
        node (str): The current node being visited.
        visited (Set[str]): Set of nodes that have been permanently marked (fully processed).
        stack (List[str]): List representing the ordering of nodes.
        graph (Dict[str, Set[str]]): The dependency graph.
        temp_marks (Set[str]): Set of nodes that have been temporarily marked (currently being processed).

    Raises:
        ValueError: If a circular dependency is detected.
    """
    if node in temp_marks:
        raise ValueError(f"Circular dependency detected involving '{node}'.")
    if node not in visited:
        temp_marks.add(node)
        for neighbor in graph.get(node, set()):
            topological_sort_util(neighbor, visited, stack, graph, temp_marks)
        temp_marks.remove(node)
        visited.add(node)
        stack.insert(0, node)  # Prepend node to the stack


def topological_sort(graph: Dict[str, Set[str]]) -> List[str]:
    """
    Perform a topological sort on the dependency graph to determine the sampling order.

    Parameters:
        graph (Dict[str, Set[str]]): The dependency graph.

    Returns:
        List[str]: A list of parameter names in the order they should be sampled.

    Raises:
        ValueError: If a circular dependency is detected.
    """
    visited: Set[str] = set()
    temp_marks: Set[str] = set()
    stack: List[str] = []
    for node in graph:
        if node not in visited:
            topological_sort_util(node, visited, stack, graph, temp_marks)
    return stack


def sample_parameters(
    param_dict: Dict[str, Tuple[Any, Any]], sample_size: int
) -> Dict[str, np.ndarray]:
    """
    Sample parameters uniformly within specified bounds, respecting any dependencies.

    Parameters:
        param_dict (Dict[str, Tuple[Any, Any]]): Dictionary mapping parameter names to their bounds.
        sample_size (int): Number of samples to generate.

    Returns:
        Dict[str, np.ndarray]: A dictionary mapping parameter names to arrays of sampled values.

    Raises:
        ValueError: If dependencies cannot be resolved due to missing parameters or circular dependencies.
    """
    graph = build_dependency_graph(param_dict)
    try:
        sampling_order = topological_sort(graph)
    except ValueError as e:
        raise ValueError(f"Error in topological sorting: {e}")

    samples: Dict[str, np.ndarray] = {}
    for param in sampling_order:
        # print('sampling :', param)
        bounds = param_dict.get(param)
        if bounds is None:
            # If the parameter wasn't in the param_dict (could be a dependency only), skip it.
            continue
        lower, upper = bounds

        # Resolve bounds if they are dependent on other parameters
        if isinstance(lower, str):
            if lower in samples:
                lower = samples[lower]
            else:
                raise ValueError(
                    f"Parameter '{lower}' must be defined before '{param}'."
                )
        if isinstance(upper, str):
            if upper in samples:
                upper = samples[upper]
            else:
                raise ValueError(
                    f"Parameter '{upper}' must be defined before '{param}'."
                )

        # Ensure lower bound is less than upper bound
        # TODO: Improve this test to not only operate on sampled but on strict checks!
        if np.any(lower >= upper):
            raise ValueError(
                f"Lower bound '{lower}' must be less than upper bound '{upper}' for parameter '{param}'."
            )

        # Ensure lower and upper are arrays of the correct size
        lower_array = np.full(sample_size, lower) if np.isscalar(lower) else lower
        upper_array = np.full(sample_size, upper) if np.isscalar(upper) else upper

        # Sample uniformly within bounds
        try:
            samples[param] = np.random.uniform(
                low=lower_array, high=upper_array
            )  #  size=)
        except ValueError as e:
            raise ValueError(f"Error sampling parameter '{param}': {e}")

    return samples


# Example usage
if __name__ == "__main__":
    # Define parameter space with dependencies
    my_parameter_space_dict = {"sv": (0, 2.5), "st": (0, "t"), "t": (0, 1)}

    sample_size = 1000  # Number of samples
    try:
        samples = sample_parameters(my_parameter_space_dict, sample_size)
    except ValueError as e:
        print(f"An error occurred: {e}")
    else:
        # Access the samples
        sv_samples = samples["sv"]
        st_samples = samples["st"]
        t_samples = samples["t"]

        # Verify that st_samples <= t_samples
        assert np.all(st_samples <= t_samples), "Constraint st <= t not satisfied"

        # Print a few samples
        print("First 5 samples:")
        for i in range(5):
            print(
                f"Sample {i+1}: sv={sv_samples[i]:.3f}, st={st_samples[i]:.3f}, t={t_samples[i]:.3f}"
            )

    # Example with more parameters
    print("\nExample with more parameters:")
    my_parameter_space_dict_extended = {
        "a": (0, 5),
        "b": (0, "a"),
        "c": ("b", "a"),
        "d": ("c", 10),
        "e": ("d", "a"),
    }

    try:
        samples_extended = sample_parameters(
            my_parameter_space_dict_extended, sample_size=10
        )
    except ValueError as e:
        print(f"An error occurred: {e}")
    else:
        # Validate constraints
        print(samples_extended)
        assert np.all(samples_extended["b"] <= samples_extended["a"])
        assert np.all(samples_extended["c"] >= samples_extended["b"])
        assert np.all(samples_extended["c"] <= samples_extended["a"])
        assert np.all(samples_extended["d"] >= samples_extended["c"])
        assert np.all(samples_extended["e"] >= samples_extended["d"])
        assert np.all(samples_extended["e"] <= samples_extended["a"])

        print("First 5 samples of extended parameters:")
        for i in range(5):
            print(
                f"Sample {i+1}: a={samples_extended['a'][i]:.3f}, b={samples_extended['b'][i]:.3f}, "
                f"c={samples_extended['c'][i]:.3f}, d={samples_extended['d'][i]:.3f}, e={samples_extended['e'][i]:.3f}"
            )

    # Example with a circular dependency
    print("\nExample with circular dependency:")
    my_parameter_space_dict_circular = {"a": ("b", 10), "b": ("a", 5)}

    try:
        samples_circular = sample_parameters(
            my_parameter_space_dict_circular, sample_size=1000
        )
    except ValueError as e:
        print(f"Circular dependency detected: {e}")

First 5 samples:
Sample 1: sv=1.513, st=0.206, t=0.353
Sample 2: sv=2.280, st=0.304, t=0.609
Sample 3: sv=0.003, st=0.182, t=0.520
Sample 4: sv=1.782, st=0.040, t=0.273
Sample 5: sv=1.330, st=0.179, t=0.327

Example with more parameters:
An error occurred: Lower bound '[8.93804328 6.26015423 8.40481852 6.54958014 9.34970262 8.83782311
 5.39782031 5.79496447 3.63425659 8.71665485]' must be less than upper bound '[3.17373276 0.96621815 3.92959482 3.29637917 1.48853162 2.51234778
 0.12912169 0.02293098 0.22544352 4.53006551]' for parameter 'e'.

Example with circular dependency:
Circular dependency detected: Error in topological sorting: Circular dependency detected involving 'b'.


In [42]:
samples

{'sv': array([1.51345565e+00, 2.27961877e+00, 2.54966198e-03, 1.78224670e+00,
        1.32980758e+00, 3.00296070e-01, 8.86922237e-01, 2.34701346e+00,
        1.04440193e+00, 6.97402619e-01, 1.26393962e+00, 9.22568576e-01,
        2.16367935e+00, 2.48157222e+00, 2.29916781e+00, 1.59975039e-01,
        1.85774566e+00, 4.00935155e-01, 1.68837455e+00, 1.04442307e+00,
        1.68083596e+00, 2.33245562e+00, 1.46429607e+00, 1.10923422e-01,
        2.00400589e+00, 2.42168327e+00, 2.12000428e+00, 1.94934597e+00,
        2.09484428e+00, 1.97963289e+00, 1.86119375e+00, 9.79225598e-01,
        1.63910493e+00, 1.65126344e+00, 7.16214047e-01, 2.02808087e+00,
        7.58886765e-01, 9.04508057e-01, 1.22048719e+00, 1.90470879e+00,
        1.22891745e+00, 9.55158115e-01, 9.81663558e-01, 2.11010394e-02,
        8.40124813e-01, 1.65888799e+00, 3.14111270e-01, 1.84069833e+00,
        1.62172239e+00, 1.72550091e+00, 1.15950362e+00, 3.68255303e-01,
        1.89469819e+00, 1.19869849e+00, 1.45005027e+00, 2.

In [24]:
my_graph = build_dependency_graph(my_parameter_space_dict)

In [25]:
my_graph

defaultdict(set, {'t': {'st'}, 'st': set(), 'sv': set()})

In [21]:
my_graph_adjusted = {"sv": set(), "st": set(), "t": {"st"}}

In [22]:
topological_sort(my_graph_adjusted)

sv visited: set()
st visited: {'sv'}
t visited: {'st', 'sv'}
  t -> st
st visited: {'st', 'sv'}


['t', 'st', 'sv']

In [ ]:
def parse_bounds(bounds: Tuple[Any, Any]) -> Set[str]:
    """
    Parse the bounds of a parameter and extract any dependencies.

    Parameters:
        bounds (Tuple[Any, Any]): A tuple containing the lower and upper bounds,
                                  which can be numeric or strings indicating dependencies.

    Returns:
        Set[str]: A set of parameter names that the bounds depend on.
    """
    dependencies = set()
    for value in bounds:
        if isinstance(value, str):
            dependencies.add(value)
    return dependencies

In [27]:
my_parameter_space_dict_extended = {
    "a": (0, 5),
    "b": (1, "a"),
    "c": ("b", "a"),
    "d": ("c", 10),
    "e": ("d", "a"),
}

my_graph_extended = build_dependency_graph(my_parameter_space_dict_extended)
# topological_sort(my_graph_extended)

In [28]:
my_graph_extended

defaultdict(set,
            {'a': {'b', 'c', 'e'},
             'b': {'c'},
             'c': {'d'},
             'd': {'e'},
             'e': set()})

In [29]:
topological_sort(my_graph_extended)

a visited: set()
  a -> e
e visited: set()
  a -> c
c visited: {'e'}
  c -> d
d visited: {'e'}
  d -> e
e visited: {'e'}
  a -> b
b visited: {'e', 'd', 'c'}
  b -> c
c visited: {'e', 'd', 'c'}


['a', 'b', 'c', 'd', 'e']

In [ ]:
def topological_sort_util(
    node: str,
    visited: Set[str],
    stack: List[str],
    graph: Dict[str, Set[str]],
    temp_marks: Set[str],
) -> None:
    """
    Helper function for performing a depth-first search in the topological sort.

    Parameters:
        node (str): The current node being visited.
        visited (Set[str]): Set of nodes that have been permanently marked (fully processed).
        stack (List[str]): List representing the ordering of nodes.
        graph (Dict[str, Set[str]]): The dependency graph.
        temp_marks (Set[str]): Set of nodes that have been temporarily marked (currently being processed).

    Raises:
        ValueError: If a circular dependency is detected.
    """

    if node in temp_marks:
        raise ValueError(f"Circular dependency detected involving '{node}'.")
    if node not in visited:
        temp_marks.add(node)
        for neighbor in graph.get(node, set()):
            topological_sort_util(neighbor, visited, stack, graph, temp_marks)
        temp_marks.remove(node)
        visited.add(node)
        stack.insert(0, node)  # Prepend node to the stack

In [17]:
my_parameter_space_dict_extended.get("a")

(0, 5)

In [44]:
import ssms

bounds_tmp = ssms.config.model_config["ddm"]["param_bounds"]
names_tmp = ssms.config.model_config["ddm"]["params"]
param_space = {
    names_tmp[i]: (bounds_tmp[0][i], bounds_tmp[1][i]) for i in range(len(names_tmp))
}

In [57]:
from ssms.support_utils import sample_parameters_from_constraints

out_theta_dict = sample_parameters_from_constraints(param_space, 1)

In [54]:
isinstance(bounds_tmp, list)

True

In [55]:
names_tmp

['v', 'a', 'z', 't']

In [56]:
names_tmp

['v', 'a', 'z', 't']

In [58]:
out_theta_dict

{'t': array([0.15013799]),
 'v': array([2.26326653]),
 'z': array([0.33250436]),
 'a': array([2.35704019])}

In [61]:
my_list = []
from functools import reduce

reduce(lambda key_: my_list.append(out_theta_dict[key_]), list(out_theta_dict.keys()))

TypeError: <lambda>() takes 1 positional argument but 2 were given

In [66]:
np.array([out_theta_dict[key_][0] for key_ in names_tmp]).astype(np.float32)

array([2.2632666 , 2.3570402 , 0.33250436, 0.15013799], dtype=float32)

In [67]:
from ssms.basic_simulators.simulator import _theta_dict_to_array

In [70]:
_theta_dict_to_array(out_theta_dict, names_tmp)

array([[2.2632666 , 2.3570402 , 0.33250436, 0.15013799]], dtype=float32)